## COD API, Matches --> explore, format, clean & reshape

Activision Call of Duty API use case for **Matches** endpoint, using a slightly amended version of callofduty.py client and custom cleaning functions (from wzkd app)

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

#### Login using SSO

In [2]:
# We're storing our SSO token in an .env file stored locally to separate our conf from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### Slightly modify client methods to call the API matches endpoint

In [3]:
import httpx
import backoff

# This time we're adding additional methods in the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use

# 1. Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# 2. following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


@backoff.on_exception(backoff.expo, httpx.HTTPError, max_time=50, max_tries=8)
async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):
    """
    Returns matches history, with username's stats for every match
    Modifications compared to callofduty.py > client.GetPlayerMatches :
     - removed if platform == 'Activision', no longer supported
     - filtered out summary data from API's result: ['data'] becomes ['data']['matches']
     - default number of matches returned is now 20 (max allowed by the API) instead of 10
     - added @backoff decorator to handle (some of) API rate/availability limits
    """
    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"][
        "matches"
    ]  # originally (in callofduty.py client), API result was truncated here



async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# 3. add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

#### bonus: loop deeper into (detailed) matches history, varying the 'endTimestamp' param

In [4]:
import itertools

async def getMoreMatchesDetailed(client, platform, username, title, mode, **kwargs):
    """
    A loop to go deeper into (detailed) matches history,
    using endTimestamp argument defined in GetMatchesDetailed()
    """

    n_calls = kwargs.get("n_calls", 2)
    all_batchs = []
    endTimestamp = 0

    while len(all_batchs) < n_calls:
        batch = await client.GetMatchesDetailed(
            platform, username, title, mode, endTimestamp=endTimestamp
        )
        endTimestamp = batch[-1]["utcStartSeconds"] * 1000
        all_batchs.append(batch)
    more_matches = list(itertools.chain(*all_batchs))

    return more_matches

# more_matches = await getMoreMatchesDetailed(client, "battle", "amadevs#1689", Title.ModernWarfare, Mode.Warzone, n_calls=3)

#### Get Matches data

##### Option: save previous result so we're not getting annoyed by API rate limits or inconsistencies -,-

## Match result : structure

In [5]:
# load previously saved data
with open('data/matches_history_20.pkl', 'rb') as f:
    matches = pickle.load(f)

### Overview : dict --> df

In [6]:
df_matches = pd.DataFrame(matches)
display(df_matches.head(2))
keys = list(df_matches.keys())
keys.sort()
pprint(keys)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1651966776,1651967982,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,3512568621842457939,1206000,None,1,wz,36,"{'kills': 35.0, 'medalXp': 1780.0, 'kdRatio': ...","{'team': 'team_six', 'rank': 54.0, 'awards': {...",17,None,False,False
1,1651965412,1651966617,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,12994993685561265204,1205000,None,1,wz,36,"{'kills': 15.0, 'medalXp': 515.0, 'kdRatio': 0...","{'team': 'team_fourteen', 'rank': 54.0, 'award...",16,None,False,False


['draw',
 'duration',
 'gameType',
 'map',
 'matchID',
 'mode',
 'player',
 'playerCount',
 'playerStats',
 'playlistName',
 'privateMatch',
 'rankedTeams',
 'teamCount',
 'utcEndSeconds',
 'utcStartSeconds',
 'version']


### API structure for one match (a 'row')

##### Reminder : matches endpoint returns n (max 20) matches & their associated stats, as 'rows', for a single --queried player

In [7]:
pprint(matches[10], depth=2)

{'draw': False,
 'duration': 1206000,
 'gameType': 'wz',
 'map': 'mp_escape4',
 'matchID': '2842410646192225510',
 'mode': 'br_dmz_playlist_wz325/rbrthbmo_quads',
 'player': {'awards': {...},
            'brMissionStats': {...},
            'clantag': 'YT.IG',
            'loadout': [...],
            'loadouts': [...],
            'rank': 54.0,
            'team': 'team_three',
            'uno': '2621859779580650696',
            'username': 'gentil_renard'},
 'playerCount': 36,
 'playerStats': {'assists': 4.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 3375.0,
                 'damageTaken': 3030.0,
                 'deaths': 25.0,
                 'distanceTraveled': 101435.09,
                 'executions': 0.0,
                 'headshots': 3.0,
                 'kdRatio': 0.48,
                 'kills': 12.0,
                 'longestStreak': 2.0,
                 'matchXp': 0.0,
                 'medalXp': 190.0,
       

### Focus : what's in 'playerStats' ?

In [8]:
player_stats = df_matches['playerStats'].apply(pd.Series)
display(player_stats.head(5))
pprint(player_stats.keys())

,kills,medalXp,kdRatio,matchXp,bonusXp,scoreXp,wallBangs,score,totalXp,timePlayed,...,longestStreak,scorePerMinute,damageDone,distanceTraveled,deaths,damageTaken,teamSurvivalTime,gulagDeaths,gulagKills,teamPlacement
0,35.0,1780.0,2.333333,4958.0,0.0,16334.0,0.0,13900.0,23072.0,1283.0,...,11.0,650.038971,9516.0,93901.234,15.0,1660.0,NaN,NaN,NaN,NaN
1,15.0,515.0,0.454545,4958.0,0.0,15899.0,0.0,6300.0,21372.0,1285.0,...,3.0,294.163424,5237.0,86782.600,33.0,3770.0,NaN,NaN,NaN,NaN
2,15.0,405.0,0.625000,0.0,0.0,6875.0,0.0,5875.0,7280.0,1397.0,...,3.0,252.326414,6200.0,94580.870,24.0,2869.0,NaN,NaN,NaN,NaN
3,28.0,655.0,1.217391,4958.0,0.0,20227.0,0.0,8375.0,25840.0,1281.0,...,4.0,392.271663,8463.0,79195.555,23.0,2735.0,NaN,NaN,NaN,NaN
4,23.0,530.0,1.210526,0.0,0.0,7650.0,0.0,7650.0,8180.0,1293.0,...,6.0,354.988399,7218.0,96789.210,19.0,2930.0,NaN,NaN,NaN,NaN


Index(['kills', 'medalXp', 'kdRatio', 'matchXp', 'bonusXp', 'scoreXp',
       'wallBangs', 'score', 'totalXp', 'timePlayed', 'headshots',
       'executions', 'assists', 'nearmisses', 'challengeXp',
       'percentTimeMoving', 'miscXp', 'rank', 'longestStreak',
       'scorePerMinute', 'damageDone', 'distanceTraveled', 'deaths',
       'damageTaken', 'teamSurvivalTime', 'gulagDeaths', 'gulagKills',
       'teamPlacement'],
      dtype='object')


In [9]:
displayed_cols = ['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']
# Just in case a specific game mode does not contain a value we wanted to display
displayed_cols = [col for col in displayed_cols if col in list(player_stats.keys())]
player_stats[displayed_cols].head(5)

,headshots,distanceTraveled,teamSurvivalTime
0,12.0,93901.234,NaN
1,1.0,86782.600,NaN
2,5.0,94580.870,NaN
3,13.0,79195.555,NaN
4,11.0,96789.210,NaN


### Focus : 'player', a nested entry

In [10]:
player = df_matches['player'].apply(pd.Series)
display(player.head(5))
pprint(player.keys())

,team,rank,awards,username,uno,clantag,loadouts,brMissionStats,loadout
0,team_six,54.0,"{'one_shot_kill': 444240.0, 'gun_butt': 0.0, '...",gentil_renard,2621859779580650696,YT.IG,"[{'primaryWeapon': {'name': 's4_ar_asierra44',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_ar_asierra44',..."
1,team_fourteen,54.0,"{'low_health_kill': 435888.0, 'gun_butt': 1106...",gentil_renard,2621859779580650696,YT.IG,"[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_aalpha12',..."
2,team_eight,54.0,"{'low_health_kill': 936336.0, 'one_shot_kill':...",gentil_renard,2621859779580650696,YT.IG,"[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_oscar12', ..."
3,team_nine,54.0,"{'one_shot_kill': 242736.0, 'revenge': 1172496...",gentil_renard,2621859779580650696,YT.IG,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ..."
4,team_five,54.0,"{'low_health_kill': 556032.0, 'one_shot_kill':...",gentil_renard,2621859779580650696,YT.IG,"[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 'iw8_sh_dpapa12', ..."


Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'loadouts',
       'brMissionStats', 'loadout'],
      dtype='object')


#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [11]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
match_index = 2
pprint(player['loadout'][match_index][0], depth=2)

{'extraPerks': [{...}, {...}, {...}],
 'killstreaks': [{...}, {...}, {...}],
 'lethal': {'image': None,
            'imageLarge': None,
            'label': None,
            'name': 'equip_throwing_knife',
            'progressionImage': None},
 'perks': [{...}, {...}, {...}],
 'primaryWeapon': {'attachments': [...],
                   'imageIcon': None,
                   'imageLoot': None,
                   'label': None,
                   'name': 'iw8_sh_oscar12',
                   'variant': '0'},
 'secondaryWeapon': {'attachments': [...],
                     'imageIcon': None,
                     'imageLoot': None,
                     'label': None,
                     'name': 's4_pi_wecho',
                     'variant': '8'},
 'tactical': {'image': None,
              'imageLarge': None,
              'label': None,
              'name': 'equip_adrenaline',
              'progressionImage': None}}


#### Inside 'player' entry, 'brMissionStats' is a (list of) dict

In [12]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
match_index = 4
pprint(player['brMissionStats'][match_index], depth=3)

{'missionStatsByType': {},
 'missionsComplete': 0,
 'totalMissionWeaponXpEarned': 0.0,
 'totalMissionXpEarned': 0.0}


## Format & clean API **matches** result using customized tools (wzkd app)

In [14]:
# import conf / parse files & methods from wzkd app
import json
import toml

sys.path.insert(0, os.path.abspath('./'))
from src.utils import load_labels, load_conf
from src.api_format import res_to_df, format_df, augment_df

In [15]:
# conf and labels files stored here as well.
# labels is needed for parsing games modes/weapons, conf stores values such as n of loadouts to extract or columns names
file_labels = "wz_labels.json"
filepath_labels = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+ "/src/" + file_labels
LABELS = load_labels(filepath_labels)
pprint(LABELS, depth=2)

file_conf = "conf.toml"
filepath_conf = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/src/" + file_conf
CONF = load_conf(filepath_conf)
pprint(CONF, depth=2)

{'missions': {'types': [...]},
 'modes': {'battle_royale': {...}, 'multiplayer': {...}},
 'weapons': {'cat_names': {...},
             'categories': [...],
             'names': {...},
             'prefixes': [...]}}
{'API_OUTPUT_FORMAT': {'float_cols': [...],
                       'int_cols': [...],
                       'n_loadouts': 3,
                       'ts_cols': [...]},
 'APP_BEHAVIOR': {'br_only': True, 'filename': {...}, 'mode': 'offline'},
 'APP_DISPLAY': {'cols': {...}, 'labels': {...}}}


In [16]:
with open('data/matches_history_20.pkl', 'rb') as f:
    res = pickle.load(f)
tmp = pd.DataFrame(res)
display(tmp.head(2))
tmp.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1651966776,1651967982,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,3512568621842457939,1206000,None,1,wz,36,"{'kills': 35.0, 'medalXp': 1780.0, 'kdRatio': ...","{'team': 'team_six', 'rank': 54.0, 'awards': {...",17,None,False,False
1,1651965412,1651966617,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,12994993685561265204,1205000,None,1,wz,36,"{'kills': 15.0, 'medalXp': 515.0, 'kdRatio': 0...","{'team': 'team_fourteen', 'rank': 54.0, 'award...",16,None,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   utcStartSeconds  20 non-null     int64 
 1   utcEndSeconds    20 non-null     int64 
 2   map              20 non-null     object
 3   mode             20 non-null     object
 4   matchID          20 non-null     object
 5   duration         20 non-null     int64 
 6   playlistName     0 non-null      object
 7   version          20 non-null     int64 
 8   gameType         20 non-null     object
 9   playerCount      20 non-null     int64 
 10  playerStats      20 non-null     object
 11  player           20 non-null     object
 12  teamCount        20 non-null     int64 
 13  rankedTeams      0 non-null      object
 14  draw             20 non-null     bool  
 15  privateMatch     20 non-null     bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 2.4+ KB


In [17]:
# flatten-expand into a DataFrame the result from COD API, for matches history
df_matches = res_to_df(res, CONF)
display(df_matches.head(5))
df_matches.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,uno,clantag,loadout_1,loadout_2,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,scavenger,timedrun
0,1651966776,1651967982,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,3512568621842457939,1206000,1,wz,36,17,...,2621859779580650696,YT.IG,"{'primaryWeapon': {'name': 's4_ar_asierra44', ...","{'primaryWeapon': {'name': 's4_ar_asierra44', ...",NaN,0,0.0,0.0,NaN,NaN
1,1651965412,1651966617,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,12994993685561265204,1205000,1,wz,36,16,...,2621859779580650696,YT.IG,"{'primaryWeapon': {'name': 'iw8_sh_aalpha12', ...","{'primaryWeapon': {'name': 'iw8_sh_aalpha12', ...","{'primaryWeapon': {'name': 's4_ar_asierra44', ...",0,0.0,0.0,NaN,NaN
2,1651963243,1651964449,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,11009559426743301282,1206000,1,wz,38,14,...,2621859779580650696,YT.IG,"{'primaryWeapon': {'name': 'iw8_sh_oscar12', '...","{'primaryWeapon': {'name': 's4_sm_tyankee100',...","{'primaryWeapon': {'name': 's4_ar_asierra44', ...",0,0.0,0.0,NaN,NaN
3,1651959818,1651961023,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,641966715710456723,1205000,1,wz,36,15,...,2621859779580650696,YT.IG,"{'primaryWeapon': {'name': 'iw8_sh_dpapa12', '...","{'primaryWeapon': {'name': 'iw8_sh_dpapa12', '...","{'primaryWeapon': {'name': 'iw8_sh_dpapa12', '...",0,0.0,0.0,NaN,NaN
4,1651958410,1651959616,mp_escape4,br_dmz_playlist_wz325/rbrthbmo_quads,7430509966098335393,1206000,1,wz,36,14,...,2621859779580650696,YT.IG,"{'primaryWeapon': {'name': 'iw8_sh_dpapa12', '...","{'primaryWeapon': {'name': 'iw8_sh_dpapa12', '...","{'primaryWeapon': {'name': 'iw8_sh_dpapa12', '...",0,0.0,0.0,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   utcStartSeconds             20 non-null     int64  
 1   utcEndSeconds               20 non-null     int64  
 2   map                         20 non-null     object 
 3   mode                        20 non-null     object 
 4   matchID                     20 non-null     object 
 5   duration                    20 non-null     int64  
 6   version                     20 non-null     int64  
 7   gameType                    20 non-null     object 
 8   playerCount                 20 non-null     int64  
 9   teamCount                   20 non-null     int64  
 10  draw                        20 non-null     bool   
 11  privateMatch                20 non-null     bool   
 12  kills                       20 non-null     float64
 13  medalXp                     20 non-nu

In [18]:
# make the stats human-readable and parse some values (weapons, games modes)
df_formatted = format_df(df_matches, CONF, LABELS)
display(df_formatted.head(5))
df_formatted.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,uno,clantag,loadout_1,loadout_2,loadout_3,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,scavenger,timedrun
0,2022-05-08 01:39:36,2022-05-08 01:59:42,mp_escape4,Rbth Pldr x4,3512568621842457939,20,1,wz,36,17,...,2621859779580650696,YT.IG,AS44 ZRG,AS44 Pila,NaN,0,0.0,0.0,<NA>,<NA>
1,2022-05-08 01:16:52,2022-05-08 01:36:57,mp_escape4,Rbth Pldr x4,12994993685561265204,20,1,wz,36,16,...,2621859779580650696,YT.IG,JAK-12 Top break,JAK-12 Top break,AS44 ZRG,0,0.0,0.0,<NA>,<NA>
2,2022-05-08 00:40:43,2022-05-08 01:00:49,mp_escape4,Rbth Pldr x4,11009559426743301282,20,1,wz,38,14,...,2621859779580650696,YT.IG,Origin Top break,Type100 PKM,AS44 Pila,0,0.0,0.0,<NA>,<NA>
3,2022-05-07 23:43:38,2022-05-08 00:03:43,mp_escape4,Rbth Pldr x4,641966715710456723,20,1,wz,36,15,...,2621859779580650696,YT.IG,R9-0 Top break,R9-0 Top break,R9-0 Top break,0,0.0,0.0,<NA>,<NA>
4,2022-05-07 23:20:10,2022-05-07 23:40:16,mp_escape4,Rbth Pldr x4,7430509966098335393,20,1,wz,36,14,...,2621859779580650696,YT.IG,R9-0 Top break,R9-0 Top break,R9-0 Top break,0,0.0,0.0,<NA>,<NA>


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   utcStartSeconds             20 non-null     datetime64[ns]
 1   utcEndSeconds               20 non-null     datetime64[ns]
 2   map                         20 non-null     object        
 3   mode                        20 non-null     object        
 4   matchID                     20 non-null     object        
 5   duration                    20 non-null     object        
 6   version                     20 non-null     int64         
 7   gameType                    20 non-null     object        
 8   playerCount                 20 non-null     int64         
 9   teamCount                   20 non-null     int64         
 10  draw                        20 non-null     bool          
 11  privateMatch                20 non-null     bool          
 

In [19]:
# build up some KPI's, for now just 'gulagStatus' (win or loss)
df_augmented = augment_df(df_formatted, LABELS)
display(df_augmented[['utcEndSeconds', 'matchID', 'mode', 'loadout_1', 'gulagKills', 'gulagDeaths', 'gulagStatus']])

,utcEndSeconds,matchID,mode,loadout_1,gulagKills,gulagDeaths,gulagStatus
0,2022-05-08 01:59:42,3512568621842457939,Rbth Pldr x4,AS44 ZRG,NaN,NaN,NaN
1,2022-05-08 01:36:57,12994993685561265204,Rbth Pldr x4,JAK-12 Top break,NaN,NaN,NaN
2,2022-05-08 01:00:49,11009559426743301282,Rbth Pldr x4,Origin Top break,NaN,NaN,NaN
3,2022-05-08 00:03:43,641966715710456723,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN
4,2022-05-07 23:40:16,7430509966098335393,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN
5,2022-05-07 23:15:21,2219002783150675048,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN
6,2022-05-07 22:52:39,15027642791380618065,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN
7,2022-05-07 22:34:13,10186179547298871152,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN
8,2022-05-07 22:08:38,1725921644460524430,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN
9,2022-05-07 21:43:54,17573637914071510539,Rbth Pldr x4,R9-0 Top break,NaN,NaN,NaN


# Tests

In [20]:
# load previously saved data
with open('data/matches_history_60.pkl', 'rb') as f: # or 'matches_60.pkl'
    more_matches = pickle.load(f)

In [21]:
df = res_to_df(more_matches, CONF)
df = format_df(df, CONF, LABELS)
df = augment_df(df, LABELS)
df.head()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,loadout_3,missionsComplete,totalMissionWeaponXpEarned,totalMissionXpEarned,assassination,domination,masterassassination,scavenger,timedrun,gulagStatus
0,2022-05-08 01:39:36,2022-05-08 01:59:42,mp_escape4,Rbth Pldr x4,3512568621842457939,20,1,wz,36,17,...,NaN,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
1,2022-05-08 01:16:52,2022-05-08 01:36:57,mp_escape4,Rbth Pldr x4,12994993685561265204,20,1,wz,36,16,...,AS44 ZRG,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
2,2022-05-08 00:40:43,2022-05-08 01:00:49,mp_escape4,Rbth Pldr x4,11009559426743301282,20,1,wz,38,14,...,AS44 Pila,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
3,2022-05-07 23:43:38,2022-05-08 00:03:43,mp_escape4,Rbth Pldr x4,641966715710456723,20,1,wz,36,15,...,R9-0 Top break,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
4,2022-05-07 23:20:10,2022-05-07 23:40:16,mp_escape4,Rbth Pldr x4,7430509966098335393,20,1,wz,36,14,...,R9-0 Top break,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
